In [1]:
%matplotlib inline
import cv2
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
import os

In [3]:
def detect_match(algorithm,query_img,train_img,min_match_count,verbose=False):
    query_image = query_img
    train_image = train_img
    key_pts1,des1 = algorithm(query_image,None)
    key_pts2,des2 = algorithm(train_image,None)
    msed=np.inf
    if not (isinstance(des1,np.float32)&isinstance(des2,np.float32)):
        des1 = np.float32(des1)
        des2 = np.float32(des2)
    
    flann_idx = 1
    index_params = dict(algorithm = flann_idx, trees = 5)
    search_params = dict(checks = 50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(des1,des2,k=2)

    # store all the good matches as per Lowe's ratio test.
    good = [m for m,n in matches if m.distance < 0.7*n.distance]
    
    if len(good)>min_match_count:
        src_pts = np.float32([ key_pts1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ key_pts2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        if M is None:
            return [0],[0],[0],[0],np.inf
        #print(M,mask)
        matchesMask = mask.ravel().tolist()
        h,w = query_image.shape
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts,M)
        msed = np.mean([np.sqrt(np.sum(diff)) for diff in (np.power(pts-dst,2))]/(np.sqrt(h**2+w**2)))
        #cv2.polylines(train_image,[np.int32(dst)],True,255,3, cv2.LINE_AA)
    else:
        if verbose==True:
            print( "Not enough matches are found - {}/{}".format(len(good), min_match_count) )
        matchesMask = [0]
    
    return key_pts1, key_pts2, good, matchesMask, msed

def plot_match(query_img, train_img, key_pts1, key_pts2, good, matchesMask):
    draw_params = dict(matchColor = (255.0,255.0,255.0), # draw matches in white color
                   singlePointColor = (255.0,0,0),
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
    img3 = cv2.drawMatches(query_img, key_pts1, train_img, key_pts2, good, None, **draw_params)
    plt.imshow(img3, 'gray'),plt.show()

In [4]:
alg_dict=dict(sift=cv2.xfeatures2d.SIFT_create().detectAndCompute,
              surf=cv2.xfeatures2d.SURF_create(200).detectAndCompute)
min_matches=10

In [5]:
data_folder1 = os.path.join(os.curdir,'trainKvadr')
data_set1 = [os.path.join(data_folder1,file) for file in os.listdir(data_folder1) if os.path.isfile(os.path.join(data_folder1,file))]
data_folder2 = os.path.join(os.curdir,'trainPony')
data_set2 = [os.path.join(data_folder2,file) for file in os.listdir(data_folder2) if os.path.isfile(os.path.join(data_folder2,file))]


In [6]:
data_set2

['./trainPony/IMG_3290.JPG',
 './trainPony/IMG_3247.JPG',
 './trainPony/IMG_3253.JPG',
 './trainPony/IMG_3326.JPG',
 './trainPony/IMG_3285.JPG',
 './trainPony/IMG_3287.JPG',
 './trainPony/IMG_3250.JPG',
 './trainPony/IMG_3244.JPG',
 './trainPony/IMG_3330.JPG',
 './trainPony/IMG_3319.JPG',
 './trainPony/IMG_3279.JPG',
 './trainPony/IMG_3241.JPG',
 './trainPony/IMG_3334.JPG',
 './trainPony/IMG_3320.JPG',
 './trainPony/IMG_3283.JPG',
 './trainPony/IMG_3295.JPG',
 './trainPony/IMG_3336.JPG',
 './trainPony/IMG_3323.JPG',
 './trainPony/IMG_3257.JPG',
 './trainPony/IMG_3218.JPG',
 './trainPony/IMG_3224.JPG',
 './trainPony/IMG_3230.JPG',
 './trainPony/IMG_3225.JPG',
 './trainPony/IMG_3232.JPG',
 './trainPony/IMG_3222.JPG',
 './trainPony/IMG_3221.JPG',
 './trainPony/IMG_3341.JPG',
 './trainPony/IMG_3210.JPG',
 './trainPony/IMG_3238.JPG',
 './trainPony/IMG_3212.JPG',
 './trainPony/IMG_3217.JPG',
 './trainPony/IMG_3216.JPG',
 './trainPony/IMG_3215.JPG',
 './trainPony/IMG_3307.JPG',
 './trainPony/

In [7]:
ponies = [cv2.imread(image,0) for image in data_set2]

In [8]:
import itertools
import time
inliers_matches = dict()
indexes = list(itertools.product(alg_dict.keys(),["inliers/matches","distance MSE","time (s)"]))
for im_j_idx in range(len(ponies)):
    inliers_matches["img"+str(im_j_idx)]=list()

In [9]:
for alg_name, alg in alg_dict.items():
    for im_i_idx, image_i in enumerate(ponies):
        time_start=time.monotonic()
        _, __, match, inlier ,msd = detect_match(algorithm=alg,query_img=image_i,train_img=ponies[13],
                                                     min_match_count=5)
        time_end=time.monotonic()
        if match==0:
            match=np.inf
        inliers_matches["img"+str(im_i_idx)].append(np.sum(inlier)/(len(match)))
        inliers_matches["img"+str(im_i_idx)].append(msd)
        inliers_matches["img"+str(im_i_idx)].append(time_end-time_start)

In [10]:
import pandas as pd
index = pd.MultiIndex.from_tuples(indexes,names=['algorithm','property'])
train_df=pd.DataFrame(inliers_matches,index=index)

In [11]:
train_df

img0      img1      img2      img3      img4  \
algorithm property                                                            
sift      inliers/matches  0.580645  0.230769  0.200000  0.403509  0.400000   
          distance MSE     0.084266  0.574722  0.504344  0.077118  2.861696   
          time (s)         4.088559  3.950162  3.952156  4.061322  3.873130   
surf      inliers/matches  0.483721  0.235849  0.126761  0.334661  0.151515   
          distance MSE     0.084288  0.545571  0.545355  0.072932  0.607487   
          time (s)         1.349531  1.517039  1.417104  1.429524  1.355131   

                               img5       img6      img7      img8      img9  \
algorithm property                                                             
sift      inliers/matches  0.250000   0.170213  0.200000  0.397590  0.473684   
          distance MSE     0.528636   0.501116  0.513917  0.072116  0.017386   
          time (s)         3.897639   3.955687  3.963196  3.982966  3.928899   
surf      inliers/matches  0.116667   0.095238  0.086022  0.441379  0.591241   
          distance MSE     0.519367  21.816866  1.924999  0.067623  0.017447   
          time (s)         1.343258   1.513530  1.633685  1.548074  1.464180   

                           ...     img37     img38     img39     img40  \
algorithm property         ...                                           
sift      inliers/matches  ...  0.424528  0.191489  0.156250  0.575758   
          distance MSE     ...  0.099289  0.500093  0.542736  0.043088   
          time (s)         ...  4.173198  4.199039  4.129053  4.060777   
surf      inliers/matches  ...  0.387560  0.142857  0.114754  0.440000   
          distance MSE     ...  0.063955  0.543892  0.538192  0.047522   
          time (s)         ...  1.541379  1.611578  1.705072  1.405573   

                              img41     img42     img43     img44     img45  \
algorithm property                                                            
sift      inliers/matches  0.485915  0.530864  0.145833  0.561224  0.206897   
          distance MSE     0.035185  0.056140  0.518736  0.070520  0.560302   
          time (s)         4.098031  4.171344  4.030318  4.138474  3.892522   
surf      inliers/matches  0.477064  0.372470  0.103704  0.355469  0.090909   
          distance MSE     0.035264  0.058164  0.537653  0.058238  0.527510   
          time (s)         1.429665  1.430694  1.593171  1.369907  1.665102   

                              img46  
algorithm property                   
sift      inliers/matches  0.535211  
          distance MSE     0.080923  
          time (s)         4.017659  
surf      inliers/matches  0.443750  
          distance MSE     0.076777  
          time (s)         1.387382  

[6 rows x 47 columns]